In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
%cd /content/drive/MyDrive/NER-BERT

/content/drive/MyDrive/NER-BERT


In [3]:
!git clone https://github.com/huggingface/transformers
%cd transformers
!pip install seqeval

Cloning into 'transformers'...
remote: Enumerating objects: 186814, done.
remote: Counting objects: 100% (30/30), done.
remote: Compressing objects: 100% (29/29), done.
remote: Total 186814 (delta 8), reused 4 (delta 0), pack-reused 186784
Receiving objects: 100% (186814/186814), 206.59 MiB | 14.15 MiB/s, done.
Resolving deltas: 100% (131120/131120), done.
Updating files: 100% (4080/4080), done.
/content/drive/MyDrive/NER-BERT/transformers
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16162 sha256=905a908f5ee52af11992060cf3243160eb8d81554c341d5734691398f91817ef
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval


In [6]:
!git clone https://huggingface.co/dmis-lab/biobert-v1.1

Cloning into 'biobert-v1.1'...
remote: Enumerating objects: 29, done.
remote: Total 29 (delta 0), reused 0 (delta 0), pack-reused 29
Unpacking objects: 100% (29/29), 108.30 KiB | 276.00 KiB/s, done.
Filtering content: 100% (2/2), 826.39 MiB | 55.63 MiB/s, done.
fatal: cannot exec '/content/drive/MyDrive/NER-BERT/biobert-v1.1/.git/hooks/post-checkout': Permission denied


# Dataset overview and Preprocessing

**Check dataset size**

In [7]:
#!wc -l /content/drive/MyDrive/NER-BERT/Conll2003/train.txt
#!wc -l /content/drive/MyDrive/NER-BERT/Conll2003/test.txt
#!wc -l /content/drive/MyDrive/NER-BERT/Conll2003/valid.txt

!wc -l /content/drive/MyDrive/NER-BERT/data/plant-disease/pd.train
!wc -l /content/drive/MyDrive/NER-BERT/data/plant-disease/pd.test
!wc -l /content/drive/MyDrive/NER-BERT/data/plant-disease/pd.dev

14637 /content/drive/MyDrive/NER-BERT/data/plant-disease/pd.train
1689 /content/drive/MyDrive/NER-BERT/data/plant-disease/pd.test
1834 /content/drive/MyDrive/NER-BERT/data/plant-disease/pd.dev


**First 20 lines of train dataset**

In [8]:
!head -n20 /content/drive/MyDrive/NER-BERT/data/plant-disease/pd.train

studies NNS O
on IN O
magnesiums NNS O
mechanism NN O
of IN O
action NN O
in IN O
digitalis NNP B-Plant
induced VBD O
arrhythmias NNP B-Disease

the DT O
mechanism NN O
by IN O
which WDT O
magnesium NN O
affects VBZ O
digitalis NNP B-Plant
induced VBN O
arrhythmias NNP B-Disease


**Keep only words and name entity tags**

In [10]:
%cd /content/drive/MyDrive/NER-BERT/data/plant-disease/ready

/content/drive/MyDrive/NER-BERT/data/plant-disease/ready


In [ ]:
!cat /content/drive/MyDrive/NER-BERT/data/plant-disease/pd.train | cut -d " " -f 1,3 > pd-train_temp.txt
!cat /content/drive/MyDrive/NER-BERT/data/plant-disease/pd.test | cut -d " " -f 1,3 > pd-test_temp.txt
!cat /content/drive/MyDrive/NER-BERT/data/plant-disease/pd.dev| cut -d " " -f 1,3 > pd-dev_temp.txt

**Run preprocess script**

In [ ]:
!cp /content/drive/MyDrive/NER-BERT/preprocess.py /content/drive/MyDrive/NER-BERT/data/plant-disease/ready/preprocess.py

In [12]:
!python3 preprocess.py pd-train_temp.txt "biobert-base-cased-v1.2" 128 > train.txt
!python3 preprocess.py pd-test_temp.txt "biobert-base-cased-v1.2" 128 > test.txt
!python3 preprocess.py pd-dev_temp.txt "biobert-base-cased-v1.2" 128 > dev.txt

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_errors.py", line 286, in hf_raise_for_status
    response.raise_for_status()
  File "/usr/local/lib/python3.10/dist-packages/requests/models.py", line 1021, in raise_for_status
    raise HTTPError(http_error_msg, response=self)
requests.exceptions.HTTPError: 401 Client Error: Unauthorized for url: https://huggingface.co/biobert-base-cased-v1.2/resolve/main/tokenizer_config.json

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/transformers/utils/hub.py", line 385, in cached_file
    resolved_file = hf_hub_download(
  File "/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_validators.py", line 118, in _inner_fn
    return fn(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py", line 1368, in hf_hub_download
    raise head_

In [ ]:
!cat train.txt test.txt dev.txt | cut -d " " -f 2 | grep -v "^$"| sort | uniq > /content/drive/MyDrive/NER-BERT/data/plant-disease/pdrlabels.txt

# Train section

In [ ]:
!cp /content/drive/MyDrive/NER-BERT/run_ner.py /content/drive/MyDrive/NER-BERT/data/plant-disease/ready/run_ner.py
!cp /content/drive/MyDrive/NER-BERT/utils_ner.py /content/drive/MyDrive/NER-BERT/data/plant-disease/ready/utils_ner.py

In [ ]:
!python3 run_ner.py \
  --data_dir /content/drive/MyDrive/NER-BERT/data/plant-disease/ready \
  --model_type bert \
  --labels /content/drive/MyDrive/NER-BERT/data/plant-disease/pdrlabels.txt \
  --model_name_or_path "bert-base-multilingual-cased" \
  --output_dir "/content/drive/MyDrive/NER-BERT/model/plant-disease" \
  --max_seq_length  128 \
  --num_train_epochs 4 \
  --per_gpu_train_batch_size 32 \
  --save_steps 1000 \
  --logging_steps 1000 \
  --seed 43 \
  --do_train \
  --do_eval \
  --do_predict \
  --overwrite_output_dir

2024-02-23 13:04:15.185832: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-23 13:04:15.185882: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-23 13:04:15.187134: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-23 13:04:16.253278: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
02/23/2024 13:04:17 - WARNING - __main__ -   Process rank: -1, device: cuda, n_gpu: 1, distributed training: False, 16-bits training: False
02/23/2024 13:04:17 - INFO - __main__ -   Tokenizer arguments: {'do_lower_case': False}
model.safeten

In [ ]:
def read_examples_from_file(file_path):
    """Read words and labels from a CoNLL-2002/2003 data file.

    Args:
      file_path (str): path to NER data file.

    Returns:
      examples (dict): a dictionary with two keys: words (list of lists)
        holding words in each sequence, and labels (list of lists) holding
        corresponding labels.
    """
    with open(file_path, encoding="utf-8") as f:
        examples = {"words": [], "labels": []}
        words = []
        labels = []
        for line in f:
            if line.startswith("-DOCSTART-") or line == "" or line == "\n":
                if words:
                    examples["words"].append(words)
                    examples["labels"].append(labels)
                    words = []
                    labels = []
            else:
                splits = line.split(" ")
                words.append(splits[0])
                if len(splits) > 1:
                    labels.append(splits[-1].replace("\n", ""))
                else:
                    # Examples could have no label for mode = "test"
                    labels.append("O")
    return examples

In [ ]:
y_true = read_examples_from_file("/content/drive/MyDrive/NER-BERT/data/plant-disease/ready/test.txt")["labels"]
y_pred = read_examples_from_file("/content/drive/MyDrive/NER-BERT/model/plant-disease/test_predictions.txt")["labels"]

In [ ]:
from seqeval.metrics import classification_report as classification_report_seqeval

print(classification_report_seqeval(y_true, y_pred))

              precision    recall  f1-score   support

     Disease       0.74      0.74      0.74       101
       Plant       0.69      0.62      0.65        86

   micro avg       0.72      0.68      0.70       187
   macro avg       0.71      0.68      0.69       187
weighted avg       0.71      0.68      0.70       187



In [ ]:
import numpy as np
from sklearn.metrics import classification_report

print(classification_report(np.concatenate(y_true), np.concatenate(y_pred)))

              precision    recall  f1-score   support

   B-Disease       0.82      0.78      0.80       101
     B-Plant       0.81      0.73      0.77        85
   I-Disease       0.67      0.81      0.73        52
     I-Plant       0.94      0.59      0.72        29
           O       0.96      0.97      0.97      1307

    accuracy                           0.93      1574
   macro avg       0.84      0.78      0.80      1574
weighted avg       0.94      0.93      0.93      1574



In [ ]:
from transformers import pipeline, AutoModelForTokenClassification, AutoTokenizer

model = AutoModelForTokenClassification.from_pretrained("/content/drive/MyDrive/NER-BERT/TMA/NLP/bertner")
tokenizer = AutoTokenizer.from_pretrained("/content/drive/MyDrive/NER-BERT/TMA/NLP/bertner")

ner_model = pipeline('ner', model=model, tokenizer=tokenizer)

In [ ]:
sequence = "England is a country that is part of the United Kingdom"
ner_model(sequence)

[{'entity': 'B-LOC',
  'score': 0.9992632,
  'index': 1,
  'word': 'England',
  'start': 0,
  'end': 7},
 {'entity': 'B-LOC',
  'score': 0.9993463,
  'index': 10,
  'word': 'United',
  'start': 41,
  'end': 47},
 {'entity': 'I-LOC',
  'score': 0.9976726,
  'index': 11,
  'word': 'Kingdom',
  'start': 48,
  'end': 55}]